In [12]:
import geopandas as gpd
import pandas as pd
import numpy as np
import random
from scipy.spatial import distance
from deap import base, creator, tools, algorithms
import zipfile
import os
from multiprocessing import Pool

# Extract ZIP files
fqhc_zip_path = "MO_2018_Federally_Qualified_Health_Center_Locations.zip"
residential_zip_path = "Mo_pop_Sim.zip"

extract_dir_fqhc = "MO_2018_FQHC"
extract_dir_residential = "Mo_pop_Sim"

os.makedirs(extract_dir_fqhc, exist_ok=True)
os.makedirs(extract_dir_residential, exist_ok=True)

with zipfile.ZipFile(fqhc_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir_fqhc)
with zipfile.ZipFile(residential_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir_residential)

# Load data
fqhc_shapefile_path = os.path.join(extract_dir_fqhc, "MO_2018_Federally_Qualified_Health_Center_Locations", "MO_2018_Federally_Qualified_Health_Center_Locations.shp")
residential_csv_path = os.path.join(extract_dir_residential, "Mo_pop_Sim.csv")

fqhc_gdf = gpd.read_file(fqhc_shapefile_path)
residential_df = pd.read_csv(residential_csv_path)

# Convert residential DataFrame to GeoDataFrame
residential_gdf = gpd.GeoDataFrame(
    residential_df,
    geometry=gpd.points_from_xy(residential_df["long"], residential_df["lat"]),
    crs="EPSG:4326",
)

# Convert coordinates to NumPy arrays
fqhc_coords = fqhc_gdf[["Longitude", "Latitude"]].to_numpy()
residential_coords = residential_gdf[["long", "lat"]].to_numpy()

# Define max radius for accessibility (30 miles converted to meters)
MAX_DISTANCE = 30 * 1609.34  # 30 miles to meters

def compute_fitness(selection):
    total_residents_served = 0
    selected_centers = fqhc_coords[list(selection)]

    for center in selected_centers:
        distances = np.linalg.norm(residential_coords - center, axis=1)
        total_residents_served += np.sum(distances < MAX_DISTANCE)

    return (total_residents_served,)

# Define Genetic Algorithm Setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, len(fqhc_coords)-1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 8)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", compute_fitness)

# Enable multiprocessing
pool = Pool()
toolbox.register("map", pool.map)

# Run Genetic Algorithm
def run_ga(ngen=50, pop_size=30):
    pop = toolbox.population(n=pop_size)
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=True)
    best_individual = tools.selBest(pop, k=1)[0]
    return best_individual

# Run the algorithm with optimized settings
best_selection = run_ga(ngen=50, pop_size=30)
best_fqhc_locations = fqhc_gdf.iloc[list(best_selection)]

# Print selected locations
print("Selected FQHC locations for mental health services:")
print(best_fqhc_locations)

# Save output to a file
best_fqhc_locations.to_file("selected_fqhc_locations.shp")

# Close the multiprocessing pool
pool.close()
pool.join()


/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	30    
1  	23    
2  	20    
3  	24    
4  	17    
5  	23    
6  	20    
7  	7     
8  	17    
9  	13    
10 	16    
11 	15    
12 	17    
13 	18    
14 	14    
15 	17    
16 	19    
17 	18    
18 	13    
19 	18    
20 	25    
21 	17    
22 	22    
23 	21    
24 	17    
25 	19    
26 	19    
27 	17    
28 	22    
29 	17    
30 	17    
31 	11    
32 	22    
33 	16    
34 	18    
35 	19    
36 	18    
37 	15    
38 	17    
39 	20    
40 	19    
41 	10    
42 	17    
43 	21    
44 	22    
45 	20    
46 	18    
47 	15    
48 	14    
49 	15    
50 	16    
Selected FQHC locations for mental health services:
     OBJECTID                   Group_Name  \
20         21           Access Family Care   
154       155                      COMTREA   
23         24           Access Family Care   
12         13  Katy Trail Community Health   
7           8    Great Mines Health Center   
23         24           Access Family Care   
122       123    Northwest Health Services   
122     

**1.Determining the Relative Fitness of a Location**

I chose the density-based approach, where fitness is determined by the number of residents within a 30-mile radius of each FQHC. A location with a higher number of residents nearby is considered better because it can serve more people efficiently.

**2.Assumptions**

Proximity is the key factor for accessibility—residents are more likely to visit an FQHC if it is closer.
Other factors such as transportation availability, socioeconomic conditions, or service quality are not considered in this approach.

**3.Fitness Function**

The fitness function measures the number of residents within 30 miles (48.28 km) of a selected FQHC. The goal is to maximize this number.

The metric is maximized because we want to serve as many residents as possible with the selected 8 locations.


**4.Implementation Details**

**Step 1: Extract Data**

Load FQHC locations from a shapefile.

Load residential locations from a CSV file.

**Step 2: Convert Data to GeoDataFrames**

Convert latitude/longitude data into spatial objects for efficient distance calculations.

**Step 3: Compute Fitness Function**

Use NumPy vectorized operations (np.linalg.norm()) to efficiently compute distances.

Count the number of residents within 30 miles of each FQHC.

**Step 4: Optimize Using Genetic Algorithm (GA)**

Population size: 30

Generations: 50

Selection method: Tournament selection

Crossover: Two-point crossover

Mutation: Shuffle mutation

Parallel processing: multiprocessing.Pool() to speed up evaluations.

**Step 5: Output Selected Locations**

The algorithm selects 8 FQHCs with the highest accessibility.

The selected locations are printed and saved as a shapefile.

**5.Results: Selected FQHC Locations**

The selected 8 FQHC locations based on the genetic algorithm are:


**Joplin**

**Festus**

**Neosho**

**Marshall**

**Potosi**

**St. Joseph**

These FQHCs maximize coverage, ensuring that the highest number of Missouri residents have access to mental health services.


